In [1]:
import requests
import json
import numpy as np

from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

import math

In [19]:
model = SentenceTransformer('all-MiniLM-L6-v2')

characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green', 'others']

#characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green']

sentences = []

for character in characters:
    f = open(character + ".txt", "r")
    lines = f.readlines()
    sentences.append(lines)

i=0
embeddings = {}
for character in characters:
    embeddings[character] = model.encode(sentences[i])
    i+=1


#to confirm number of lines used to create the embedding
print(len(embeddings['Monica Geller']))

for character in characters:

    mean = embeddings[character][0]
    for embedding in embeddings[character]:
        mean += embedding
    
    embeddings[character] = mean



1000


In [3]:
seasons = []

for i in range(1,11):

    # load season i
    if i<10:
        json_file = 'https://raw.githubusercontent.com/emorynlp/character-mining/master/json/friends_season_0'+str(i)+'.json'
    else:
        json_file = 'https://raw.githubusercontent.com/emorynlp/character-mining/master/json/friends_season_10.json'
    
    r = requests.get(json_file)
    
    seasons.append(json.loads(r.text))

In [4]:
def tf_idf_calculator(lines):    
    # Compute the term frequency of each word in each sentence
    tf = []
    
    for sentence in lines:
        tf_sentence = {}
        for word in sentence.split():
            tf_sentence[word] = tf_sentence.get(word, 0) + 1
        tf.append(tf_sentence)

    # Compute the inverse document frequency of each word
    idf = {}
    for sentence in lines:
        for word in sentence.split():
            idf[word] = idf.get(word, 0) + 1
    for word in idf:
        idf[word] = math.log(len(lines) / idf[word])

    # Compute the tf-idf of each sentence
    tf_idf = []
    for i, sentence in enumerate(lines):
        tf_idf_sentence = 0
        for word in tf[i]:
            tf_idf_sentence += tf[i][word] * idf[word]
        tf_idf.append(tf_idf_sentence)
    
    return tf_idf

In [5]:
all_lines = []
for season in seasons:
    for episode in season['episodes']:
        for scene in episode['scenes']:
            for utterance in scene['utterances']:
                if utterance['speakers'] != []:
                    all_lines.append(utterance['transcript'])

#remove duplicates
all_lines = list( dict.fromkeys(all_lines) )

tf_idf = tf_idf_calculator(all_lines)

all_lines_tf_idf = []

#format: all_lines_tf_idf = [...,[tf-idf score ,sentence],...]
for i,sentence in enumerate(all_lines):
    cell = []
    cell= [tf_idf[i], sentence]
    all_lines_tf_idf.append(cell)
    
# lower -> higher     
sorted_list = sorted(all_lines_tf_idf)

tf_idfs_utterances = []

#get only the strings
for elem in sorted_list:
    tf_idfs_utterances.append(elem[1])


#for elem in sorted_list:
#    print(elem)


In [6]:
def compute_similarity(vector):

    angles = {}

    for character in characters:
        angles[character] = cosine_similarity(vector.reshape(1,-1), embeddings[character].reshape(1,-1))[0][0]

    
    
    #Smaller angles between vectors produce larger cosine values, indicating greater cosine similarity

    character = [i for i in angles if angles[i]==max(angles.values())]


    return character[0]


all_sentences = []

for s in sentences:
    for l in s:
        all_sentences.append(l)


In [33]:
predicted = []
real = []

correct = 0
total = 0

for season in seasons:
    for episode in season['episodes']:
        for scene in episode['scenes']:
            for utterance in scene['utterances']:
                if len(utterance['speakers']) == 1 and utterance['transcript'] not in all_sentences:

                    total += 1

                    line_embed = model.encode(utterance['transcript'])
    
                    pred = compute_similarity(line_embed)

                    predicted.append(pred)
                    real.append(utterance['speakers'][0])

                    if pred == utterance['speakers'][0]:
                        correct+=1
                    elif pred == "others" and utterance['speakers'][0] not in ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green']:
                        correct+=1

                    #print("predicted :", pred)
                    #print("real:", utterance['speakers'][0])

                    
print(accuracy_score(real, predicted)*100)

accuracy = (float(correct)/total)*100

print(accuracy)

9.156117452467125
15.238360381900659


classifier WITH filter in the test set

In [18]:
predicted = []
real = []

correct = 0
total = 0


threshold = 15

for season in seasons:
    for episode in season['episodes']:
        for scene in episode['scenes']:
            for utterance in scene['utterances']:
                if len(utterance['speakers']) == 1 and utterance['transcript'] not in all_sentences:

                    index = tf_idfs_utterances.index(utterance['transcript'])

                    if sorted_list[index][0] >= 20:

                        total += 1

                        line_embed = model.encode(utterance['transcript'])
        
                        pred = compute_similarity(line_embed)

                        predicted.append(pred)
                        real.append(utterance['speakers'][0])

                        if pred == utterance['speakers'][0]:
                            correct+=1
                        elif pred == "others" and utterance['speakers'][0] not in ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green']:
                            correct+=1

                        #print("predicted :", pred)
                        #print("real:", utterance['speakers'][0])

                    
print(accuracy_score(real, predicted)*100)

accuracy = (float(correct)/total)*100

print(accuracy)

20.27194355535827
20.27194355535827


In [ ]:

51367